[RNN-1(LSTM)](https://brunch.co.kr/@chris-song/9)<br>
[RNN-2(LSTM)](https://deeplearning4j.org/kr/lstm)<br>
아래 내용은 상기 링크를 정리한 것.

### Contents
* [일반적인 인공 신경망](#Feedforwoard Networks)
* [RNNs](#rnn)
* [BPTT](#BPTT)
* [Vanshing(and Exploding) Gradients](#vg)

# Long Short-Term Memory(LSTM)

## Feedforword Networks(일반적인 인공 신경망)<a name="Feedforwoard Networks"></a>

RNNs을 이해하려면 우선 일반적인 인공 신경망(FFNets)를 이해 해야한다. <br>
일반적인 인공 신경망을 Feed-forward neural networks라고도 하는데,<br>
그 이름에서 이미 Recurrent neural networks(RNNs)와 어떤 점이 다른지 드러난다.<br>
FFNets은 데이터를 입력하면 연산이 입력층에서 은닉층(hidden layers)를 거쳐 출력까지 진행된다.<br>
이 과정에서 입력 데이터는 모든 노드를 딱 한번 지나가게 된다.<br>
그러나 RNNs은 은닉층의 결과가 다시 같은 은닉층의 입력으로 들어가도록 연결 되어있다.<br>
<br>
FFNets의 입/출력이 각 사진과 사진의 라벨(고양이, 강아지 등등) 이라면(Supervised학습일 경우),<br>
이 FFNets은 사진에 있는 물체에서 패턴을 파악해서 적절한 라벨을 찾아준다.<br>
아래 그림의 FFNets의 구조를 참고.<br>
![FFNets](https://deeplearning4j.org/img/feedforward_rumelhart.png)

FFNets은 라벨을 붙여놓은 이미지 데이터로 학습을 진행하면서 점점 오차를 줄여간다.<br>
학습이 이루어지기 전에 데이터의 일부를 따로 관리하는데,<br>
이를 테스트 셋이라고 한다.<br>
테스트 셋은 학습과정에서는 사용하지 않는다.<br>
비유하면 문제유형은 알려주지만 출제할 문제를 알려주지 않는것 과 같다.<br>
한편 신경망은 학습 과정에서 사용하는 데이터를 독립적으로 학습한다.<br>
즉, 데이터의 순서는 중요하지 않다.<br>
<br>
다시 말해 FFNets은 시간 순서를 무시하고 현재 주어진 데이터만 가지고 판단한다.<br>
즉, 이 데이터 전에 봤던 데이터가 무엇인지 기억하지 않는다.<br>

## RNNs<a name = 'rnn'></a>

RNNs은 FFNets과는 조금 다르다.<br>
RNNs는 현재 들어온 입력 데이터와 과거에 입력 받았던 데이터를 동시에 고려한다.<br>
아래의 [Elman](https://web.stanford.edu/group/pdplab/pdphandbook/handbookch8.html)이 제안한 아주 간단한 RNNs의 구조도를 보면,<br> 
입력으로 [B, T, S, X, P, E]가 들어오는데 은닉층에서는 이 입력데이터와 좌측 하단의 [CONTEXT UNIT]을 다 입력으로 받는다.<br>
![Elman](https://deeplearning4j.org/img/srn_elman.png)

과거의 출력이 다시 입력되는 구조를 피드백 구조라한다.<br>

아래 그림을 통해 더 자세하게 보면
![RNN](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

$x_{t-1}$시점의 RNNs출력값은 $x_t$시점의 RNNs 출력값에도 영향을 준다.<br>
피드백 구조는 방금 일어난 출력을 다시 입력으로 집어 넣는데,<br>
이 구조 덕분에 RNNs는 기억능력이 있다고 한다.<br>
인공 신경망이 기억 능력을 넣는 이유는 일반 적인 인공신경망이 할 수 없는 일을 하기 위해서 이다.<br>
즉, 배열의 정보를 보유하고 이 정보를 이용해 원하는 분석을 하는 것이다.<br>
<br>
배열의 정보는 $S$(hidden state)에 저장되며,<br>
이 hidden states에 저장된 정보는 시간이 지난 후에 저장정보가 필요한 입력이 들어오면 다시 사용한다.<br>
마치 머릿속에 기억을 저장하고 있듯이 RNNs는 Hidden States에 데이터를 저장하고 있다.<br>
이제 위의 작동 과정을 수식으로 살펴본다.<br>

$$
S_t = \phi(Ux_t + WS_{t-1})
$$

* 시간 $t$에서 은닉층의 상태, 즉 hidden state가 갖고 있는 값을 $h_t$라고 한다.<br>
* 이 값은 같은 시점 $t$에 들어온 입력 $x_t$와 계수 행렬$U$,
* 시간 $t-1$에서 hidden state 값 $S_{t-1}$,
* 그리고 $S_{t}$ 와 $S_{t-1}$의 관계를 나타낸 행렬 $W$의 함수이다.<br>
($W$행렬은 Markov체인의 상태 전이 행렬(transition matrix)과 비슷하다.
* 계수 $U$는 지금 들어온 입력과 보유하고 있던 기억(hidden state value)이 얼마나 중요한지 판단 하는 값이다.
* 예를 들어 $U$가 아주 큰값이 경우 기억하고 있는 $S$는 별로 중요하지 않고,<br> 
현재 들어온 입력값($x_t$)를 위주로 판단을 내린다.
* FFNets과 마찬가지로 출력단에서 오차를 계산하고 이 오차는 다시 hidden state으로 내려오는데,<br>
그 값을 기준으로 $U$를 업데이트 한다.
* 입력 $x$와 기억$S$의 합은 함수$\phi$를 통과하면서 압축된다.<br>
보통 tanh나 Logisic sigmoid(로지스틱 시그모이드)를 사용한다.<br>
이 함수는 출력값의 범위를 제한해주면서 전 구간에서 미분이 가능하기 때문에 Backprop이 잘 적용된다.

* 여기에서 $S_t$와 $S_{t-1}$의 피드백은 매순간마다,<br>
즉, 모든 $t$마다 이루어진다.<br>
    - 그런데 $S_t$의 값을 구하기 위해선 $S_{t-1}$이 필요하고, <br> 
    - $S_{t-1}$값을 구하기 위해서는 다시 $S_{t-2}$가 필요하다.<br>
    - 결과적으로 hidden state는 $S$를 전부 기억하고 있어야 하는데,<br> 
    실제로 값을 무한히 저장할 수 없으므로 사용 가능한 메모리 등 여러 상황에 맞추어 적당한 범위까지만 저장을 한다.

* 예를 들어 RNNs이 문자열을 입력받으면, 우선 첫 번째 문자로 학습을 하고 학습한 내용을 hidden state층에 저장한다.<br>
그리고 이 값을 이용해 두 번째 입력을 처리한다.<br>
예를 들면 $q$ 다음엔 $u$가 올 확률이 높고, $t$다음엔 $h$가 올 확률이 높은데, 이 정보를 이용할 수 있다.<br>
<br>

RNNs은 아래 시각화로 쉽게 이해가 가능하다.
![RNNs](https://i.imgur.com/kpZBDfV.gif)

에니메이션에서 
* $x$는 입력, 
* $w$는 입력 데이터 $x$에 곱해지는 가중치, 
* $a$는 hidden state의 활성값(=입력과 hidden state 값을 고려해 구해지는 값) 
* $b$는 hidden state가 sigmoid 함수(Activate Function)을 통과한 출력이다.

## BPTT: Backpropagation Throught Time(시간을 거슬러 가는 Backporop)<a name = 'BPTT'></a>

RNNs의 목적은 배열(시계열) 데이터를 분류하는것이다.<br>
그리고 RNNs의 학습은 다른 인공 신경망의 학습과 마찬가지로 오차의 backprop과 경사 하강법(Gradient Descent)을 사용한다.<br>
<br>
FFNets의 backprop는 출력단에서 구한 오차를 신경망을 거슬러 가면서 각 계수를 업데이트 하는것 이다.<br>
여기서 각 계수가 출력단의 오차에 얼마나 기여하는지를 계산하고 그 기여도 만큼 업데이트를 한다.<br>
기여도는 편미분 $\partial E / \partial w$으로 계산한다.<br>
그리고 이 값을 사용해 각 계수를 얼마나 증가 혹은 감소시킬지 결정한다.<br>
<br>
RNNs은 backprop의 확장판인 [BTPP(Backpropagation Through Time)](https://www.techleer.com/articles/185-backpropagation-through-time-recurrent-neural-network-training-technique/)을 사용해 계수를 학습한다.<br>
본질적으로 BTPP는 기본적인 backprop와 같다.<br>
다만 RNNs의 구조가 시간에 따라 연결되어 있기 때문에 backprop역시 시간을 거슬러 올라가며 적용되는 것이 다르다.<br>

<br>
인공 신경망은 RNNs이든 FFNets이든 결국 여러 함수의 조합이다.<br>
f(g(h(x)))와 같은 계산을 어떤 식으로 수행하는지만 다를 뿐이고,<br>
여기에 시계열 데이터를 위해 피드백이 추가되더라도 본질은 연쇄법칙(Chain Rule)에 따른 backprop이다.<br>


### Truncated BPTT: 단기 BPTT
Truncated BPTT는 시간 전체를 거슬러 올라가는 BPTT를 간략화 한 것이다.<br>
시계열 데이터가 길어지면 은닉층에 저장해야하는 양이 계속 늘어나기 때문에 모든 시간에 대한 은닉층의 값을 저장하는 것은 현식적으로 불가능 하다.<br>
따라서 적당한 선에서 타협을 한 것이 바로 단기 BPTT이다.<br>
단기 BPTT를 사용하면 기준 길이보다 오래된 값은 반영되지 않으므로 RNNs의 기억력이 짧아지는 문제가 있다.

# 그라디언트 안정화 문제(Vanishing (and Exploding) Gradients) <a name = "vg"></a>

1980년대 이미 RNNs에 대한 논문이 나오고<br>
1990년대 초반 Vanishing Gradient(그라디언트 소실)이라는 문제가 나타났다.<br>
Gradient의 개념은 아주 단순하다.<br>
두점의 기울기를 이용하여 변화량의 오차를 알아내는 것이다.<br>
즉, 점과 아래 A의 차이를 이용하여 오차를 줄여가는 것이다.<br>

![Vanishing Gradients](https://mathspace-production-media.mathspace.co/media/upload/images/001_Chapter_Entries/Rates_of_Change/rates10.png)

즉, x-y 평면에 직선을 그으면, 직선의 미분값은 정의 따라 x의 변화량에 따른 y의 변화량의 비율을 나타낸다.<br>
위 그림에서의 변화량은 A이다.<br>
<br>
이를 인공 신경망에 적용하면 모든 가중치와 오차의 관계를 구할 수 있다.<br>
즉, 신경망의 값을 얼만큼 변화시키면 그 결과로 오차가 어떻게 변하는지를 알아낼 수 있다.<br>
따라서 Gradient의 계산은 아주 중요하다.<br>
만일 Gradient를 잘 구할 수 없다면 계수와 오차의 관계를 알 수가 없고, 결과적으로 학습이 잘 되지 않는다.<br>
<br>
RNNs은 시간을 거슬러 올라가며 과거 은닉값을 추적한다.<br>
그런데 이 추적이 이어질수록(즉, 과거로 많이 거슬러 올라가면) Gradient의 계산이 잘되지 않는 경우가 있다.<br>
이유는 신경망이 곱하기 연산을 기반으로 이루어져 있기 때문이다.<br>
<br>
은행의 적금 상품을 예로 들어 1보다 아주 조금만 큰 값을 여러번 곱하면 나중에 큰 값이 되는 이를 복리의 바법이라고도 하며,<br>
적은 이율로도 오랜 기간을 보관하면 나중에 큰 금액이 된다.<br>
<br>
마찬가지로 0보다 아주 살짝 큰 값이라도 지속적으로 곱하게 되면 나중에는 1에 가까운 값이 된다.<br>
<br>
인공 신경망의 연산도 많은 곱하기로 이루어져 있고,<br>